# **Setup**

In [ ]:
import logging

logging.basicConfig(level=logging.ERROR)

In [ ]:
!pip3 install "cassio>=0.1.0"

In [ ]:
import os
from getpass import getpass
TRACELOOP_API_KEY = getpass("Please enter your TRACELOOP API Key: ")
os.environ['TRACELOOP_API_KEY'] = TRACELOOP_API_KEY
#print(os.getenv('TRACELOOP_API_KEY'))

In [ ]:
import llama_index

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Document,
    StorageContext,
)
from llama_index.indices.query.query_transform import HyDEQueryTransform
from llama_index.query_engine.transform_query_engine import TransformQueryEngine
from llama_index.vector_stores import CassandraVectorStore
from IPython.display import Markdown, display

database connection parameters and secrets

In [ ]:
try:
    from google.colab import files
    IS_COLAB = True
except ModuleNotFoundError:
    IS_COLAB = False

# Your database's Secure Connect Bundle zip file is needed:
if IS_COLAB:
    print('Please upload your Secure Connect Bundle zipfile: ')
    uploaded = files.upload()
    if uploaded:
        astraBundleFileTitle = list(uploaded.keys())[0]
        ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
    else:
        raise ValueError(
            'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
        )
else:
    # you are running a local-jupyter notebook:
    ASTRA_DB_SECURE_BUNDLE_PATH = input("Please provide the full path to your Secure Connect Bundle zipfile: ")

DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5674948432) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5674944976) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:f03b8bff-1861-4185-8d91-4e5dc70ee886
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5674946128) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:2c823767-4e6e-4c32-ba6f-fed79fd419f1
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5673795216) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Received options response on connection (5674948432) from b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = getpass("Please provide your Database Token ('AstraCS:...' string): ")

DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5495214800) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:f03b8bff-1861-4185-8d91-4e5dc70ee886
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5495216976) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5495219152) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:2c823767-4e6e-4c32-ba6f-fed79fd419f1
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5492303824) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:f03b8bff-1861-4185-8d91-4e5dc70ee886
DEBUG:cassandra.connection:Received options response on connection (5495214800) from b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:f03b8bff-1861-4185-8

In [77]:
ASTRA_DB_KEYSPACE = input("Please provide the Keyspace name for your Database: ")

DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5672848592) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5672849616) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:f03b8bff-1861-4185-8d91-4e5dc70ee886
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5672851792) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:2c823767-4e6e-4c32-ba6f-fed79fd419f1
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5427602064) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Received options response on connection (5672848592) from b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

#Establish Connectivity
cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
    },
    auth_provider=PlainTextAuthProvider(
        "token",
        ASTRA_DB_APPLICATION_TOKEN,
    ),
)

session = cluster.connect()
keyspace = ASTRA_DB_KEYSPACE

In [ ]:
OPENAI_API_KEY = getpass("Please enter your OpenAI API Key: ")

In [ ]:
import openai

openai.api_key = OPENAI_API_KEY

Creating and populating the Vector Store

In [81]:
# load documents
documents = SimpleDirectoryReader("/Users/mangat.rai/datastax/codesample/vector/paul_graham").load_data()
print(f"Total documents: {len(documents)}")
print(f"First document, id: {documents[0].doc_id}")
print(f"First document, hash: {documents[0].hash}")
print(
    f"First document, text ({len(documents[0].text)} characters):\n{'='*20}\n{documents[0].text[:500]} ..."
)

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 1


Total documents: 1
First document, id: 1f3c2ae9-4f77-4025-b67f-bf1eb3305ae0
First document, hash: 2e2d9629223c077019a6dde689049344ff2293d6c52372871420119ec049f25c
First document, text (75014 characters):


What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processi ...


Initialize the Cassandra Vector Store

Creation of the vector store entails creation of the underlying database table if it does not exist yet:

In [83]:
cassandra_store = CassandraVectorStore(
    session=session,
    keyspace=keyspace,
    table="cassandra_vector_table_1",
    embedding_dimension=1536,
)

DEBUG:llama_index.vector_stores.cassandra:Creating the Cassandra table
DEBUG:urllib3.connectionpool:https://api.traceloop.com:443 "POST /v1/traces HTTP/1.1" 200 24
DEBUG:urllib3.connectionpool:https://api.traceloop.com:443 "POST /v1/traces HTTP/1.1" 200 24


Now wrap this store into an index LlamaIndex abstraction for later querying:


In [ ]:
storage_context = StorageContext.from_defaults(vector_store=cassandra_store)

index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5674948432) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5674944976) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:f03b8bff-1861-4185-8d91-4e5dc70ee886
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5674946128) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:2c823767-4e6e-4c32-ba6f-fed79fd419f1
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5673795216) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Received options response on connection (5674948432) from b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b

Note that the above from_documents call does several things at once: it splits the input documents into chunks of manageable size (“nodes”), computes embedding vectors for each node, and stores them all in the Cassandra Vector Store.

# **Querying the store**

Basic querying

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3)
response = query_engine.query("Why did the author choose to work on AI?")
print(response.response)

DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5673795216) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Received options response on connection (5673795216) from b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5495214800) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:f03b8bff-1861-4185-8d91-4e5dc70ee886
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5495216976) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5495219152) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:2c823767-4e6e-4c32-b

### **MMR-based queries**

The MMR (maximal marginal relevance) method is designed to fetch text chunks from the store that are at the same time relevant to the query but as different as possible from each other, with the goal of providing a broader context to the building of the final answer:

In [14]:
query_engine = index.as_query_engine(vector_store_query_mode="mmr", mmr_prefetch_k = 3)
response = query_engine.query("Why did the author choose to work on AI?")
print(response.response)

The author chose to work on AI because they believed that it was a field that held the potential for creating intelligent programs. They were fascinated by the idea of teaching a program like SHRDLU to understand natural language and saw it as a way to push the boundaries of what was possible in computer science.


# **HyDE Query Transform**

HyDE stands for Hypothetical Document Embeddings, is a technique used in semantic search to find documents based on similarities in semantic embedding. It’s a zero-shot learning technique, meaning it can make predictions about data it has not been trained on.

In the context of search, HyDE works by generating a hypothetical answer to a query using a language model. This hypothetical answer is then embedded into a vector space, similar to how real documents are embedded. When a search query comes in, similar real documents are retrieved based on vector similarity to the hypothetical document. This allows for a more precise and relevant retrieval of documents, even when the exact terms used in the search query may not be present in the documents.

The aim of HyDE is to improve the quality of search results by focusing on the underlying intent of the search query, rather than just the exact words used. This makes it particularly useful for tasks like question-answering, where the goal is to find the most relevant information to answer a user’s question, rather than just finding documents that contain the exact words used in the question

### First, we query without transformation: The same query string is used for embedding lookup and also summarization.

In [15]:
query_str = "what did paul graham do after going to RISD"

In [16]:
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>After going to RISD, Paul Graham's actions are not mentioned in the given context.</b>

### Now, we use HyDEQueryTransform to generate a hypothetical document and use it for embedding lookup.

In [17]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>After going to RISD, Paul Graham's actions are not mentioned in the given context information.</b>

In this example, HyDE improves output quality significantly, by hallucinating accurately what Paul Graham did after RISD (see below), and thus improving the embedding quality, and final output.

In [ ]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]
display(Markdown(f"<b>{hyde_doc}</b>"))

# **Connecting to an existing store**

Since this store is backed by Cassandra, it is persistent by definition. So, if you want to connect to a store that was created and populated previously, here is how:

In [16]:
new_store_instance = CassandraVectorStore(
    session=session,
    keyspace=keyspace,
    table="cassandra_vector_table_1",
    embedding_dimension=1536,
)

# Create index (from preexisting stored vectors)
new_index_instance = VectorStoreIndex.from_vector_store(vector_store=new_store_instance)

# now you can do querying, etc:
query_engine = new_index_instance.as_query_engine(similarity_top_k=5)
response = query_engine.query("What did the author study prior to working on AI?")

print(response.response)


Prior to working on AI, the author studied computer science in grad school.


# **Removing documents from the index**

First get an explicit list of pieces of a document, or “nodes”, from a Retriever spawned from the index:

In [23]:
retriever = new_index_instance.as_retriever(
    vector_store_query_mode="mmr",
    similarity_top_k=3,
    vector_store_kwargs={"mmr_prefetch_factor": 4},
)
nodes_with_scores = retriever.retrieve(
    "What did the author study prior to working on AI?"
)

print(f"Found {len(nodes_with_scores)} nodes.")
for idx, node_with_score in enumerate(nodes_with_scores):
    print(f"    [{idx}] score = {node_with_score.score}")
    print(f"        id    = {node_with_score.node.node_id}")
    print(f"        text  = {node_with_score.node.text[:90]} ...")

Found 0 nodes.


Print nodes ref_doc_id they all should have same as we inserted only one record.

In [24]:
print("Nodes' ref_doc_id:")
print("\n".join([nws.node.ref_doc_id for nws in nodes_with_scores]))

Nodes' ref_doc_id:



If you need to remove the text file you uploaded:

In [ ]:
new_store_instance.delete(nodes_with_scores[0].node.ref_doc_id)

Repeat the very same query and check the results now. You should see no results being found:

In [26]:
nodes_with_scores = retriever.retrieve(
    "What did the author study prior to working on AI?"
)

print(f"Found {len(nodes_with_scores)} nodes.")

Found 0 nodes.


# **Metadata filtering**

The Cassandra vector store support metadata filtering in the form of exact-match key=value pairs at query time.

In this demo, a single source document is loaded (the paul_graham_essay.txt text file). we will attach some custom metadata to the document to illustrate how we can can restrict queries with conditions on the metadata attached to the documents.

In [ ]:
md_storage_context = StorageContext.from_defaults(
    vector_store=CassandraVectorStore(
        session=session,
        keyspace=keyspace,
        table="cassandra_vector_table_2_md",
        embedding_dimension=1536,
    )
)

def my_file_metadata(file_name: str):
    """Depending on the input file name, associate a different metadata."""
    if "essay" in file_name:
        source_type = "essay"
    elif "dinosaur" in file_name:
        # this (unfortunately) will not happen in this demo
        source_type = "dinos"
    else:
        source_type = "other"
    return {"source_type": source_type}


# Load documents and build index
md_documents = SimpleDirectoryReader(
    "/Users/mangat.rai/datastax/codesample/vector/paul_graham", file_metadata=my_file_metadata
).load_data()
md_index = VectorStoreIndex.from_documents(
    md_documents, storage_context=md_storage_context
)

DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5742125008) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5742122832) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:f03b8bff-1861-4185-8d91-4e5dc70ee886
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5729154640) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:2c823767-4e6e-4c32-ba6f-fed79fd419f1
DEBUG:cassandra.connection:Sending options message heartbeat on idle connection (5730379984) b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:432a9432-577d-4222-b785-4faf40272fd6
DEBUG:cassandra.connection:Received options response on connection (5742122832) from b2d68f45-7d3e-4936-a50c-b0742b34b3f5-us-east-2.db.astra.datastax.com:29042:f03b8bff-1861-4185-8

you can now add filtering to your query engine:

In [ ]:
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

md_query_engine = md_index.as_query_engine(
    filters=MetadataFilters(
        filters=[ExactMatchFilter(key="source_type", value="essay")]
    )
)
md_response = md_query_engine.query("How long it took the author to write his thesis?")
print(md_response.response)


To test that the filtering is at play, try to change it to use only "dinos" documents… there will be no answer